In [20]:
import numpy as np
import itertools
import pandas as pd

In [21]:
# --------- Problem setup ---------
n = 5

mu = np.array([0.12, 0.10, 0.15, 0.09, 0.11])      # expected return per share
prices = np.array([10, 12, 8, 15, 7])             # cost per share
B = 50                                            # budget
lam = 0.3                                        # risk aversion λ

# Example positive semidefinite covariance matrix
Sigma = np.array([
    [0.04, 0.01, 0.00, 0.00, 0.01],
    [0.01, 0.05, 0.01, 0.00, 0.00],
    [0.00, 0.01, 0.06, 0.02, 0.00],
    [0.00, 0.00, 0.02, 0.07, 0.01],
    [0.01, 0.00, 0.00, 0.01, 0.03]
])

upper_bounds = [5, 5, 5, 5, 5]   # brute-force search range


In [22]:
# --------- Brute force search ---------

best_x = None
best_value = -1e18
records = []

for x in itertools.product(*(range(ub + 1) for ub in upper_bounds)):
    x = np.array(x)
    cost = prices @ x
    
    if cost <= B:
        value = mu @ x - lam * (x.T @ Sigma @ x)
        records.append([*x, cost, value])
        
        if value > best_value:
            best_value = value
            best_x = x.copy()

print("===== Optimal Solution =====")
print("Best x =", best_x)
print("Objective value =", best_value)

===== Optimal Solution =====
Best x = [2 0 2 0 2]
Objective value = 0.5800000000000001


In [ ]:
from pyscipopt import Model, quicksum
import numpy as np

model = Model("mean_variance_mip")

# ----------- 1. 决策变量：整数股数 -----------------
x = [model.addVar(vtype="I", lb=0, ub=upper_bounds[i], name=f"x{i}") for i in range(n)]

# ----------- 2. budget constraint -----------------
model.addCons(quicksum(prices[i] * x[i] for i in range(n)) <= B)

# ----------- 3. 引入一个 objvar 作为线性目标 -----------------
objvar = model.addVar(vtype="C", name="objvar")

# ----------- 4. 构建线性项 μᵀx -----------
linear_term = quicksum(mu[i] * x[i] for i in range(n))

# ----------- 5. 构建二次项 xᵀΣx（SCIP 作为非线性表达式支持） -----------
quadratic_term = quicksum(Sigma[i, j] * x[i] * x[j] for i in range(n) for j in range(n))

# ----------- 6. 添加非线性约束： objvar >= μᵀx − λ·xᵀΣx -----------
# （即 objvar - linear_term + lam * quadratic_term >= 0）
model.addCons(objvar - linear_term + lam * quadratic_term <= 0)

# ----------- 7. 目标：最大化 objvar（线性的，SCIP 允许） -----------
model.setObjective(objvar, sense="maximize")

# ----------- 8. 求解 -----------------
model.optimize()

# ----------- 9. 输出结果 -----------------
print("===== SCIP MIP Nonlinear Constraint Version =====")
sol = model.getBestSol()

if sol:
    x_opt = np.array([sol[x[i]] for i in range(n)], dtype=int)
    print("Optimal x =", x_opt)
    print("Objective value =", model.getObjVal())
    print("Total cost =", prices @ x_opt)
    print("Expected return =", mu @ x_opt)
    print("Variance =", x_opt @ Sigma @ x_opt)
else:
    print("No solution found.")


===== SCIP MIP Nonlinear Constraint Version =====
Optimal x = [2 0 2 0 2]
Objective value = 0.58
Total cost = 50
Expected return = 0.76
Variance = 0.6


In [27]:
# Continous Version

model = Model("mean_variance_mip")

# ----------- 1. 决策变量：连续股数 -----------------
x = [model.addVar(vtype="C", lb=0, ub=upper_bounds[i], name=f"x{i}") for i in range(n)]

# ----------- 2. budget constraint -----------------
model.addCons(quicksum(prices[i] * x[i] for i in range(n)) <= B)

# ----------- 3. 引入一个 objvar 作为线性目标 -----------------
objvar = model.addVar(vtype="C", name="objvar")

# ----------- 4. 构建线性项 μᵀx -----------
linear_term = quicksum(mu[i] * x[i] for i in range(n))

# ----------- 5. 构建二次项 xᵀΣx（SCIP 作为非线性表达式支持） -----------
quadratic_term = quicksum(Sigma[i, j] * x[i] * x[j] for i in range(n) for j in range(n))

# ----------- 6. 添加非线性约束： objvar >= μᵀx − λ·xᵀΣx -----------
# （即 objvar - linear_term + lam * quadratic_term >= 0）
model.addCons(objvar - linear_term + lam * quadratic_term <= 0)

# ----------- 7. 目标：最大化 objvar（线性的，SCIP 允许） -----------
model.setObjective(objvar, sense="maximize")

# ----------- 8. 求解 -----------------
model.optimize()

# ----------- 9. 输出结果 -----------------
print("===== SCIP MIP Nonlinear Constraint Version =====")
sol = model.getBestSol()

if sol:
    x_opt = np.array([sol[x[i]] for i in range(n)], dtype=float)
    print("Optimal x =", x_opt)
    print("Objective value =", model.getObjVal())
    print("Total cost =", prices @ x_opt)
    print("Expected return =", mu @ x_opt)
    print("Variance =", x_opt @ Sigma @ x_opt)
else:
    print("No solution found.")


===== SCIP MIP Nonlinear Constraint Version =====
Optimal x = [1.11263812 0.         2.45358193 0.         2.74928048]
Objective value = 0.594378204991329
Total cost = 49.99999999999999
Expected return = 0.8039747169043787
Variance = 0.6986577813789827
